# Clasificación de secuencias genómicas

## Temas:
1. Introducción
  - Objetivos
  - Antecedentes y usos
  - Datos genómicos

2. Preprocesamiento básico de secuencias genómicas
  - Resumen de secuencias genómicas y su representación
  - Transformación y extracción de características (tokenización)
  - Actividad: Preparación de un conjunto de datos

3. Modelos de aprendizaje automático para clasificación de secuencias
  - Métodos clásicos de aprendizaje automático (machine learning)
  - Aprendizaje profundo (deep learning)
  - Actividad: Construcción y entrenamiento de modelos

4. Técnicas avanzadas para la clasificación de secuencias
  - Introducción a modelos de aprendizaje profundo con PyTorch
  - *Transfer learning* con HuggingFace
  ---

# Introducción

## Objetivos:
1. Entender la importancia del proceso de clasificación de secuencias
2. Entender el proceso de preparación de conjuntos de datos para la clasificación de secuencias genómicas
3. Comprender las distintas estrategias disponibles para la clasificación de secuencias
4. Implementar un algoritmo de clasificación de secuencias codificantes vs. no codificantes exitosamente

## Antecedentes
El término **genoma** se refiere a todo el material genético heredable de un ser vivo. La principal diferencia entre la genética y la genómica es el objetivo de estudio. El primero estudia **genes** individuales, así como sus productos y consecuencias de su variación. Mientras que la genómica se centra en todos los elementos que se encuentran en el genóma y cómo estos interaccionan [1]. La rama de la **genómica funcional** estudia cómo los genes y las regiones intergénicas, afectados por el ambiente resultan en fenotipos específicos, es decir, las características visibles de un organismo (color de piel, de ojos, enfermedades manifestadas, etc.) [2].

<figure>
<img    src="images/dna_double_helix.jpg"
        width=500
        height=500>
<figcaption>Fig. 1: Imagen generada de la doble hélice de la estructura del ADN.
</figure>

Tradicionalmente la clasificación de secuencias era un proceso predominantemente estadístico, sin embargo, en los últimos años el uso de herramientas de aprendizaje automático, específicamente de aprendizaje profundo, han servido para identificar y clasificar numerosos elementos genómicos. Por ejemplo, elementos como promotores, potenciadores, factores de transcripción, entre otros, han sido de mucha utilidad para comprobar la efectividad de estas novedosas estrategias [3].

<figure>
<img    src="images/IA_ML_DL.png"
        width=500
        height=500>
<figcaption>Fig. 2: Diagrama de Venn del paradigma de inteligencia artificial.
</figure>

En este taller veremos cuál es el proceso para la clasificación de secuencias genómicas desde el minado y preporcesamiento de los datos, hasta el uso de herramientas más avanzadas. De manera general, este procedimiento se resume en los siguientes pasos:

1. Entender el problema
2. Adquisición y/o búsqueda de datos
3. Pre-procesamiento de datos
4. Extracción de características
5. Elección de modelo o arquitectura
6. Entrenamiento del modelo
7. Evaluación del modelo
8. Despliegue del modelo (en aplicaciones prácticas e industriales)

<figure>
<img    src="images/ml_workflow.png"
        width=1200
        height=200>
<figcaption>Fig. 3: Flujo de trabajo convencional en el uso de herramientas de ML en aplicaciones biomédicas [4].
</figure>

---

# Preprocesamiento básico de secuencias genómicas

In [4]:
# Importaremos algunas librerías que nos serán de mucha utilidad
import torch
import torch.nn as nn
import numpy as np
from Bio import SeqIO
import gzip
import sklearn
from genomic_benchmarks.data_check import list_datasets

## Secuencias codificantes vs. no codificantes
El conjunto de datos con el que trabajaremos está disponible en la base de datos pública de Ensembl, disponible [aquí](https://ftp.ensembl.org/pub/release-111/fasta/homo_sapiens/).

In [8]:
# Cambiar paths hacia google drive para la implementación en Colab
ensembl111_path = "/media/msr/DATA/Databases/Ensembl/111/"
ncrna_path = ensembl111_path + "ncrna/Homo_sapiens.GRCh38.ncrna.fa.gz"
cdna_path = ensembl111_path + "cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz"

In [9]:
# Leemos el archivo que contiene las secuencias con las que trabajeremos
with gzip.open(ncrna_path, 'rt') as file_handler:
    ncrna_seq = [record for record in SeqIO.parse(file_handler, "fasta")]

print(ncrna_seq[:5])

[SeqRecord(seq=Seq('GTACTTATTTCAACAGCACATATTTTAAATTGGATCAATACAGAGCAGATAAGC...TTG'), id='ENST00000616830.1', name='ENST00000616830.1', description='ENST00000616830.1 ncrna scaffold:GRCh38:KI270744.1:51009:51114:-1 gene:ENSG00000278625.1 gene_biotype:snRNA transcript_biotype:snRNA gene_symbol:U6 description:U6 spliceosomal RNA [Source:RFAM;Acc:RF00026]', dbxrefs=[]), SeqRecord(seq=Seq('ATACTTACCTGGCAGGGCAGATACCATGATCTTAAAGGCAGTTTTCCCAGGGCA...GTA'), id='ENST00000612925.1', name='ENST00000612925.1', description='ENST00000612925.1 ncrna scaffold:GRCh38:KI270750.1:148668:148843:1 gene:ENSG00000277374.1 gene_biotype:snRNA transcript_biotype:snRNA gene_symbol:U1 description:U1 spliceosomal RNA [Source:RFAM;Acc:RF00003]', dbxrefs=[]), SeqRecord(seq=Seq('GTGCCTGCTTTGGCGGCACATATCCTAAAATTGGAACAATACAGAGAAAGTTAG...TTG'), id='ENST00000616468.1', name='ENST00000616468.1', description='ENST00000616468.1 ncrna scaffold:GRCh38:GL000213.1:127840:127946:-1 gene:ENSG00000278793.1 gene_biotype:snRNA transcri

In [10]:
# En este caso, la única información que nos interesa es sólo la secuencia, por lo tanto debemos especificarlo
with gzip.open(ncrna_path, 'rt') as file_handler:
    ncrna_seq = [str(record.seq)for record in SeqIO.parse(file_handler, "fasta")]

print(ncrna_seq[:5])

['GTACTTATTTCAACAGCACATATTTTAAATTGGATCAATACAGAGCAGATAAGCATGGTTACTGCCTAGGGATGGCACACAAATTCAGAAAGCATTCCATATTTTG', 'ATACTTACCTGGCAGGGCAGATACCATGATCTTAAAGGCAGTTTTCCCAGGGCAAGGCTTATCCATTCCACTCTGGATCCATCATAGGGATATGCTGATCCCTGGAATTGCCCCAAATGTGGGAAGCTCTACTGCAAAATTTTTGGTAGTGAGCGATGGCATTATGCATTCATGTA', 'GTGCCTGCTTTGGCGGCACATATCCTAAAATTGGAACAATACAGAGAAAGTTAGCATGGCTTCTGCATAAGGAGGCAGCACAACTCTTTGAGGCATTCCATATTTTG', 'CACTTTACCTGGCAGGGGAGAGACCGTGGTCACGAAGGGGGTTCTCCCAGAGTGAAGCTTCTTCATCGCACTCTAGAGTTGCTGATTCCTGTGATTTCCTCCATGTGGGAAACGGTGTTTGTGCTAGAAGAGGCTGCGCTCTTT', 'ATACTTACCTGGCAGGGGAGATACCATGATCACGAAGGTGGTTTTCCCAGGGCGAGGCTTATCCATTGCACTCCGGATGTGCTGACCCCTGCGATTTCCCCAAATGTGGGAAACTCGACTGCATAATTTGTGGTAGTGGGGGACTGCGTTCGCGCTTTCCCCTG']


**Hasta aquí, solo hemos extraído las secuencias *no codificantes*. Repetiremos el mismo proceso con las secuencias *codificantes*.**

In [11]:
# Extraemos únicamente las secuencias del conjunto de datos de secuencias codificantes
with gzip.open(cdna_path, 'rt') as file_handler:
    cdna_seq = [str(record.seq) for record in SeqIO.parse(file_handler, "fasta")]

print(cdna_seq[:5])

['GAAATAGT', 'ACTGGGGGATACG', 'CCTTCCTAC', 'GGGACAGGGGGC', 'GGGACAGGGGGC']


# Referencias

[1] Clark, D. P., Pazdernik, N. J., & McGehee, M. R. (2019). Molecular biology. Elsevier.

[2] EMBL-EBI (2024) Functional genmics 1. Disponible en: https://www.ebi.ac.uk/training/online/courses/functional-genomics-i-introduction-and-design/

[3] Grešová, K., Martinek, V., Čechák, D., Šimeček, P., & Alexiou, P. (2023). Genomic benchmarks: a collection of datasets for genomic sequence classification. BMC Genomic Data, 24(1), 25.

[4] Angermueller, C., Pärnamaa, T., Parts, L., & Stegle, O. (2016). Deep learning for computational biology. Molecular systems biology, 12(7), 878.